In [1]:
import os
import requests
import pandas as pd
import unidecode
from tqdm import tqdm

In [2]:
data = pd.read_csv('Preprocessing_Data_Laptop.csv')
data.head(100)

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,laptop,17.990.000 ₫,38% Off,3.0K Đã bán,(666),Hồ Chí Minh,28.990.000 ₫,10.200 ₫,94%,98%,100%,28,5,10,17,606
1,laptop,7.223.999 ₫,"52% Off\nLazFlash Sale,kết thúc sau 9h",51 Đã bán,(5),Hải Phòng,13.899.999 ₫,Miễn phí,100%,100%,80%,0,0,0,0,5
2,laptop,39.000 ₫,17% Off,312 Đã bán,(67),Hồ Chí Minh,47.000 ₫,17.000 ₫,96%,98%,90%,11,1,3,5,47
3,laptop,7.254.959 ₫,45% Off,16 Đã bán,(3),Hải Phòng,13.199.999 ₫,Miễn phí,100%,100%,80%,0,0,0,0,3
4,laptop,4.390.000 ₫,"20% Off\nLazFlash Sale,kết thúc sau 9h",126 Đã bán,(29),Hà Nội,5.900.000 ₫,Miễn phí,98%,100%,100%,0,0,1,0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,laptop,1.200.000 ₫,48% Off,24 Đã bán,(11),Hồ Chí Minh,2.300.000 ₫,Miễn phí,91%,100%,100%,0,1,0,1,9
96,laptop,6.000.000 ₫,NaN,136 Đã bán,(42),Nam Định,NaN,56.800 ₫,99%,100%,100%,0,0,0,0,42
97,laptop,27.590.000 ₫,16% Off,147 Đã bán,(3),Bắc Ninh,32.990.000 ₫,10.200 ₫,94%,98%,100%,0,0,0,0,4
98,laptop,27.290.000 ₫,3% Off,10 Đã bán,NaN,Hồ Chí Minh,32.999.000 ₫,9.700 ₫,94%,98%,100%,0,0,0,0,0


In [3]:
s = requests.Session()

api_base = "https://api.endpoints.anyscale.com/v1"
# Replace with long-lived credentials for production
token = "esecret_nlqldkrl2baeg63bhqpx7ap89i"
url = f"{api_base}/chat/completions"

In [4]:
def call_api(formatted_data):
    body = {
        "model": "meta-llama/Meta-Llama-3-70B-Instruct",
        "messages": [
            {
                "role": "user",
                "content": "Bạn là 1 trợ lí ảo về AI và có nhiệm vụ chính là chuẩn hóa data 1 cách chính xác nhất"
            },
            {
                "role": "assistant",
                "content": "Nice to meet you! As a virtual assistant specializing in AI, my primary responsibility is to ensure the accuracy and standardization of data. This is a crucial task, as high-quality data is essential for training reliable AI models and making informed decisions.\n\nTo achieve this goal, I'll employ a range of techniques and tools to preprocess, transform, and validate data. Here are some of the methods I'll use:\n\n1. **Data Cleaning**: I'll identify and correct errors, inconsistencies, and inaccuracies in the data, ensuring that it's complete, valid, and reliable.\n2. **Data Normalization**: I'll transform data into a consistent format, making it easier to compare and analyze. This includes converting data types, handling missing values, and performing feature scaling.\n3. **Data Standardization**: I'll apply standardized templates and formats to ensure data consistency across different sources and systems.\n4. **Data Validation**: I'll verify data against a set of rules, constraints, and business logic to ensure it meets the required standards and formats.\n5. **Data Enrichment**: I'll augment data by adding relevant information from external sources, such as APIs, databases, or other datasets, to enhance its value and accuracy.\n6. **Data Quality Metrics**: I'll track"
            },
            {
                "role": "user",
                "content": "Mỗi mục dữ liệu được ngăn cách bởi dấu |. Không bỏ bất kì 1 mục nào, không thêm bất kỳ mục nào, hãy chuẩn hóa thông tin về dạng số tương ứng, địa điểm thành không dấu, nếu là giá trị None thì ghi là None. Ngăn cách các giá trị bằng dấu phẩy, lưu ý: 52% Off\\\\nLazFlash Sale,kết thúc sau 9h (Đây là cùng 1 column), không cần giải thích output.         \nlaptop | 7.223.999 ₫ | 52% Off\\\\nLazFlash Sale,kết thúc sau 9h | 51 Đã bán| (5)| Hải Phòng | 13.899.999 ₫ |Miễn phí |100%| 100% |80%| 0| 0| 0| 0| 5\""
            },
            {
                "role": "assistant",
                "content": "Here is the standardized data:\n\nlaptop,7223999,52,51,5,Hai Phong,13899999,100,100,80,0,0,0,0,5"
            },
            {
                "role": "user",
                "content": f"{formatted_data}"
            },
        ],
        "temperature": 0,
        "max_tokens": 256,
        "top_p": 1,
        "frequency_penalty": 0
    }
    response = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body)
    return response.json()

In [5]:
def format_row(row):
    return '|'.join(map(str, row.values))

results = []
for index, row in tqdm(data.iterrows(), total=data.shape[0], desc="Processing rows"):
    formatted_data = format_row(row)
    api_response = call_api(formatted_data)
    cleaned_data = api_response['choices'][0]['message']['content'].strip()
    #print(cleaned_data)
    cleaned_data = cleaned_data.replace('Here is the standardized data:\n\n', '').replace('\n', ',')
    print(cleaned_data)
    results.append(cleaned_data.split(','))

Processing rows:   0%|          | 1/1200 [00:04<1:23:31,  4.18s/it]

laptop,17990000,38,3000,666,Ho Chi Minh,28990000,10200,94,98,100,28,5,10,17,606


Processing rows:   0%|          | 2/1200 [00:08<1:22:59,  4.16s/it]

laptop,7223999,52,51,5,Hai Phong,13899999,None,100,100,80,0,0,0,0,5


Processing rows:   0%|          | 3/1200 [00:11<1:11:43,  3.60s/it]

...

laptop,12990000,None,None,1,Nghệ An,None,90400,100,100,100,0,0,0,0,1


Processing rows: 100%|█████████▉| 1199/1200 [56:23<00:02,  2.63s/it]

laptop,5900000,14,None,None,Hanoi,6900000,None,97,100,100,0,0,0,0,0


Processing rows: 100%|██████████| 1200/1200 [56:26<00:00,  2.82s/it]

laptop,6299000,3,nan,0,Dong Nai,6500000,100,95,100,100,0,0,0,0,1


In [12]:
cleaned_df = pd.DataFrame(results, columns=data.columns)
cleaned_df

,Type,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
0,laptop,17990000,38,3000,666,Ho Chi Minh,28990000,10200,94,98,100,28,5,10,17,606
1,laptop,7223999,52,51,5,Hai Phong,13899999,None,100,100,80,0,0,0,0,5
2,laptop,39000,17,312,67,Ho Chi Minh,47000,17000,96,98,90,11,1,3,5,47
3,laptop,7254959,45,16,3,Hai Phong,13199999,100,100,80,0,0,0,0,3,None
4,laptop,4390000,20,126,29,Hà Nội,5900000,98,100,100,0,0,1,0,28,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,laptop,17200000,None,None,None,Hanoi,None,78950100,None,None,None,0,0,0,0,0
1196,laptop,19990000,None,None,None,Ho Chi Minh,None,None,99,None,100,0,0,0,0,0
1197,laptop,12990000,None,None,1,Nghệ An,None,90400,100,100,100,0,0,0,0,1
1198,laptop,5900000,14,None,None,Hanoi,6900000,None,97,100,100,0,0,0,0,0


In [13]:
cleaned_df.to_csv("Data_Laptop_After_LLMs.csv", index=False)